## Non-thermal power and bpow parameters
In this work was fitting the non-thermal emission with a broken powerlaw in photon-space, i.e
\begin{equation}
I(\epsilon)\propto\begin{cases}
    \epsilon^{-\gamma_1}, & \text{if $\epsilon<\epsilon_\mathrm{B}$}\\
    \epsilon^{-\gamma_2}, & \text{if $\epsilon\geq\epsilon_\mathrm{B}$}
  \end{cases} \quad\mathrm{[photon\;s^{-1}\;keV^{-1}\;cm^{-2}]}
\end{equation}

where $\epsilon_\mathrm{B}$ is the break in the powerlaw, and $\gamma_1=1.5$ to represent the flattening at lower energies, and $\gamma_2$ the powerlaw index of interest, which for a thick-target model should be related to the electron distribution powerlaw index $\delta=\gamma_2+1$. Here the issue is that we do not know the low energy cutoff of the electron distribution $E_\mathrm{C}$ and first attempt is just using $E_\mathrm{C}=\epsilon_\mathrm{B}$.

For the non-thermal power calculation need the photon flux [photons $\mathrm{s}^{-1}$ $\mathrm{keV}^{-1}$ $\mathrm{cm}^{-2}$] of the powerlaw with index above the break at 1 keV, which we will call $A$, then
\begin{equation}
P(\geq E_\mathrm{C})=9.5\times10^{24}\gamma_2^2 (\gamma_2-1)\beta\left(\gamma_2-\frac{1}{2},\frac{3}{2}\right)AE_\mathrm{C}^{(1-\gamma_2)}\quad\mathrm{[erg\;s^{-1}]}
\end{equation}

where $\beta(p,q)$ is the Beta function. Note that the $9.5\times10^{24}$ was used in [Hannah et al. 2008](https://doi.org/10.1086/529012) and [Lin et al. 2001](https://doi.org/10.1086/323270) - actually [Lin 1974](https://doi.org/10.1007/BF00240886) has a factor of $3\times10^{24}$ quoting numerical errors from [Brown 1971](https://doi.org/10.1007/BF00149070). However Aschwanden 2004 (book, §13.2.2 derivation and Eqn 13.2.31, based off Brown 1971 number) and subsequently papers like [Aschwanden et al. 2019](https://doi.org/10.3847/1538-4357/ab2cd4) use $4.3\times10^{24}$ Which is correct - something to do with assumption about mean molecular mass or abundance, or cross section approximations? Of course now would just fit with the thick-target model of a powerlaw electron distribution $F(\geq E_\mathrm{C})\propto E^{-\delta}$ via the Holman code. So can numerically check all this - simulate a thick-target spectrum, fit with bpow and calculate the power and compare to what it show be.

OSPEX fitted a broken powerlaw with bpow and the normalisation parameter is the photon flux of the powerlaw from below the break at the epivot of 50 keV, i.e. $I_1(50)$. So need to convert that to the photon flux at the break - as there the two powerlaws match, i.e. $I_1(\epsilon_\mathrm{B})=I_2(\epsilon_\mathrm{B})$ - and then use that value to work out the photon flux of the powerlaw index above the break at 1 keV, i.e. $I_2(1)\equiv A$. 

For the powerlaw below the break we assume some scaling constant $K_1$, so $I_1(\epsilon)=K_1\epsilon^{-1.5}$ then for the normalisation value from bpow $I_1(50)$ we get
\begin{equation}
I_1(50)=K_1 50^{-1.5}\quad \Rightarrow\quad K_1=I_1(50)/50^{-1.5}
\end{equation}
Then at the break energy the two power laws match, i.e.
\begin{equation}
I_1(\epsilon_\mathrm{B})=I_2(\epsilon_\mathrm{B})\quad \Rightarrow\quad I_1(50)\left(\frac{\epsilon_\mathrm{B}}{50}\right)^{-1.5}=K_2 \epsilon_\mathrm{B}^{-\gamma_2}\quad \Rightarrow\quad K_2=I_1(50)\left(\frac{\epsilon_\mathrm{B}}{50}\right)^{-1.5}\epsilon_\mathrm{B}^{\gamma_2}
\end{equation} 
Hence $I_2(1)\equiv A$ is going to be
\begin{equation}
I_2(1)=K_2 1^{-\gamma_2}=I_1(50)\left(\frac{\epsilon_\mathrm{B}}{50}\right)^{-1.5}\epsilon_\mathrm{B}^{\gamma_2} 1^{-\gamma_2}=I_1(50)\left(\frac{\epsilon_\mathrm{B}}{50}\right)^{-1.5}\epsilon_\mathrm{B}^{\gamma_2} \equiv A
\end{equation}

---
More straightforward to get the non-thermal power when you have the electron distribution parameters directly, say via fitting a thick-target model with parameters of the total electron flux $N$ [in electrons $\mathrm{s}^{-1}$], of a powerlaw with index $\delta$ above a low energy cutoff of $E_\mathrm{C}$ [in keV]. Which is an electron distribution of $F(E\geq E_\mathrm{C})\propto E^{-\delta}=K_n E^{-\delta}$, where $K_n$ is just a constant. Then
\begin{equation}
N=\int_{E_\mathrm{C}}^\infty F(E) dE=K_n\int_{E_\mathrm{C}}^\infty E^{-\delta}  dE\quad\Rightarrow\quad K_n=N(\delta-1)E_\mathrm{C}^{(1-\delta)}
\end{equation}

Hence the power can be found to be
\begin{equation}
P\int_{E_\mathrm{C}}^\infty E F(E) dE=N(\delta-1)E_\mathrm{C}^{(1-\delta)}\int_{E_\mathrm{C}}^\infty E^{(1-\delta)} dE
\end{equation}
which gives us
\begin{equation}
P(E\geq E_\mathrm{C})=N \frac{(\delta-1)}{(\delta-2)}E_\mathrm{C}\;[\mathrm{keV\;s}^{-1}]=1.6\times10^{-9}\times N \frac{(\delta-1)}{(\delta-2)}E_\mathrm{C}\;[\mathrm{erg\;s}^{-1}]
\end{equation}


In [1]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from sunpy.io.special import read_genx
from astropy.time import Time

from scipy.special import beta

import warnings
warnings.simplefilter('ignore')

matplotlib.rcParams['font.size']=14

In [2]:
# Just load in one of the files for checking
r05=read_genx('wee_2024_vth52_bpow.genx')

# ph flux of powerlaw below break at 50 keV, I_1(50)
I5005=np.array([r["OSX_P"][3] for r in r05["SAVEGEN0"]])
# neg powerlaw index below the break -> fixed at 1.5
g105=np.array([r["OSX_P"][4] for r in r05["SAVEGEN0"]])
# broken powerlaw break energy [keV]
eb05=np.array([r["OSX_P"][5] for r in r05["SAVEGEN0"]])
# neg powerlaw index above the break
g205=np.array([r["OSX_P"][6] for r in r05["SAVEGEN0"]])

# # Calculate A
def calc_A(I50,g1,eb,g2):
    return I50*(eb/50.)**(-1*g1)*eb**(g2)

# # Calculate the non-thermal power, use the 9.5 factor for now....
def nn9pow(A,Ec,g2):
    return 9.5e24*g2**2*(g2-1)*beta(g2-0.5,1.5)*A*Ec**(1-g2)

A05=calc_A(I5005,g105,eb05,g205) 
nn9p05=nn9pow(A05,eb05,g205)

To try and confirm which scaling number is correct for the photon powerlaw approx to the thick-target have simulated the thick-target emission from a powerlaw of electrons ($N,E_\mathrm{C},\delta$) then loaded into OSPEX, fitted with a broken powerlaw and obtained the parameters ($I_1(50),\epsilon_\mathrm{B},\gamma$). Then can calculate the power two ways and comapre which scaling parameter is more likely to be correct ($9.5$ or $4.3$ $\times10^{24}$)